# Bagel Bot Prototype Code

Jupyter notebook to explore slackclient API and develop bagel-bot functionality
Can also be used to run bot manually

In [59]:
#!pip install --upgrade slackclient

In [83]:
import os
from slack import WebClient
from slack.errors import SlackApiError
import json
import pandas as pd
from os import path
from datetime import datetime as dt

In [75]:
bagel_token = 'xxx'
my_session = WebClient(token=bagel_token)

In [62]:
def get_channel_dict(client): 
    channel_dict = {}
    response = client.conversations_list(limit=4000, exclude_archived=True)
    channels = response['channels']
    for channel in channels:
        channel_dict[channel['name']] = channel['id']
    return channel_dict

In [72]:
# channel_id for test channel
my_channel = 'bagel_test'
my_channel_id = 'xxxxxxxx'

In [64]:
def get_user_df(client, channel_id):
    user_info_list: Any = []

    response = client.conversations_members(channel=channel_id, limit=100)
    user_list = response['members']
    for user in user_list:
        response = client.users_info(user=user, include_locale=True)
        user_info_list += [response['user']]

    user_df = pd.DataFrame(user_info_list)[['id', 'name', 'real_name', 'tz']]
    user_df = user_df[(~user_df.name.str.contains('donut')) &
                      (~user_df.name.str.contains('bagel'))].reset_index(drop=True)

    return user_df

In [65]:
df = get_user_df(client, channel_id)

In [86]:
#df.head()

In [67]:
def create_matches(user_df, history_file):

    if path.exists(history_file):
        history_df = pd.read_csv(history_file)
    else:
        history_df = pd.DataFrame()

    # Match across timezones and with those they haven't matched with yet
    possible_cases_df = pd.DataFrame(columns=['name1', 'name2', 'times_paired', 'is_diff_tz'])
    user_list = user_df['name'].tolist()

    for i in range(len(user_list)):
        name1 = user_df['name'][i]
        for j in range(i + 1, len(user_list)):
            name2 = user_df['name'][j]

            if len(history_df) > 0:
                tmp_hist_df = history_df[((history_df['name1'] == name1) &
                                          (history_df['name2'] == name2)) |
                                         ((history_df['name2'] == name1) &
                                          (history_df['name1'] == name2))]
                times_paired = len(tmp_hist_df)
            else:
                times_paired = 0

            name1_mask = user_df['name'].values == name1
            name2_mask = user_df['name'].values == name2

            name1_tz = user_df[name1_mask]['tz'].values[0]
            name2_tz = user_df[name2_mask]['tz'].values[0]

            is_diff_tz = (name1_tz != name2_tz)

            possible_cases_df = possible_cases_df.append({'name1': name1,
                                                          'name2': name2,
                                                          'times_paired': times_paired,
                                                          'is_diff_tz': is_diff_tz}, ignore_index=True)

    possible_cases_df['match_strength'] = (possible_cases_df['is_diff_tz'] * 2) - possible_cases_df['times_paired']
    filter_cases_df = possible_cases_df.copy(deep=True)

    match_df = pd.DataFrame(columns=['name1', 'name2'])
    ind = 0
    for user in user_df['name'].tolist():
        top_user_match = filter_cases_df[(filter_cases_df['name1'] == user) |
                                         (filter_cases_df['name2'] == user)].sort_values('match_strength',
                                                                                         ascending=False).reset_index(
            drop=True)[['name1', 'name2']].head(1).reset_index(drop=True)
        if len(top_user_match.index) > 0:
            name1 = top_user_match.name1.values[0]
            name2 = top_user_match.name2.values[0]
            match_df.loc[ind] = [name1, name2]
            filter_cases_df = filter_cases_df[(filter_cases_df['name1'] != name1) &
                                              (filter_cases_df['name2'] != name1)]
            filter_cases_df = filter_cases_df[(filter_cases_df['name1'] != name2) &
                                              (filter_cases_df['name2'] != name2)]
            ind += 1

    # Find if anyone wasn't matched, make a second match with their top option
    for user in user_df['name'].tolist():
        tmp_match_df = match_df[(match_df['name1'] == user) |
                                (match_df['name2'] == user)]
        if len(tmp_match_df.index) == 0:
            print(f'User: {user} was not matched. Setting a second match up for them...')
            top_user_match = possible_cases_df[(possible_cases_df['name1'] == user) |
                                               (possible_cases_df['name2'] == user)].sort_values('match_strength',
                                                                                                 ascending=False).reset_index(
                drop=True)[['name1', 'name2']].head(1).reset_index(drop=True)

            name1 = top_user_match.name1.values[0]
            name2 = top_user_match.name2.values[0]
            match_df.loc[ind] = [name1, name2]

    today = dt.strftime(dt.now(), "%Y-%m-%d")
    match_df['match_date'] = today

    return match_df


def update_history(match_df, history_file):

    if path.exists(history_file):
        history_df = pd.read_csv(history_file)
    else:
        history_df = pd.DataFrame()

    history_df = pd.concat([history_df, match_df])
    history_df.to_csv(history_file, index=False)

In [68]:
def post_matches(client, user_df, match_df, my_channel_id):

    for i in range(len(match_df)):
        user1 = match_df[match_df.index == i].name1.values[0]
        user2 = match_df[match_df.index == i].name2.values[0]
        user1_id = user_df[user_df['name'] == user1]['id'].values[0]
        user2_id = user_df[user_df['name'] == user2]['id'].values[0]
        response = client.conversations_open(users=[user1_id, user2_id], return_im=True)
        conv_id = response['channel']['id']
        response = client.chat_postMessage(channel=conv_id,
                                           text=f'Hello <@{user1_id}> and <@{user2_id}>! Welcome to a new round of Bagel-Bot! Please use this DM channel to set up time to connect!',
                                           as_user='@bagel-bot')

    # Send pairings to the ds_donut channel
    response = client.chat_postMessage(channel=my_channel_id,
                                       text='The new round of pairings are in! You should have received a DM from bagel-bot with your new Bagel partner. Please post a photo here of your chat. Chat, chat away!',
                                       as_user='@bagel_bot')
    for i in range(0, len(match_df.index)):
        user1 = match_df[match_df.index == i].name1.values[0]
        user2 = match_df[match_df.index == i].name2.values[0]
        user1_id = user_df[user_df['name'] == user1]['id'].values[0]
        user2_id = user_df[user_df['name'] == user2]['id'].values[0]
        response = client.chat_postMessage(channel=my_channel_id,
                                           text=f'<@{user1_id}> and <@{user2_id}>',
                                           as_user='@bagel-bot')


In [79]:
post_matches = False
my_history_file = "my_channel_history.csv"

In [87]:
#print(f"Getting users in {my_channel} channel...")
my_user_df = get_user_df(my_session, my_channel_id)
num_users = len(my_user_df)
#print(f"{num_users} users found!")

In [88]:
#my_user_df.head()

In [ ]:
print(f"Generating optimal matches, this could take some time...")
match_df = create_matches(my_user_df, my_history_file)

print("The following matches have been generated:")
#match_df.head()

In [85]:
if post_matches:
    print(f"Posting matches to {my_channel} channel.")
    print("Setting up DM channels for matched pairs.")
    post_matches(my_session, my_user_df, match_df, my_channel_id)

    print("Updating history.")
    update_history(match_df, my_history_file)